In [2]:
#create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

24/04/08 17:31:58 WARN Utils: Your hostname, Abhisheks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.22 instead (on interface en0)
24/04/08 17:31:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/08 17:31:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/08 17:31:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path_to_data = "/Users/abhishekshah/Desktop/Legal project/small_search/split_file_155.jsonl"
documents_df = spark.read.json(path_to_data)

In [4]:
#show the schema of the data
documents_df.printSchema()

root
 |-- casebody: struct (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- attorneys: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- head_matter: string (nullable = true)
 |    |    |-- judges: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- opinions: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- author: string (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- parties: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- status: string (nullable = true)
 |-- citations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cite: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- court: struct (nullable = true)
 |   

In [5]:
#show the first 5 rows of the data
documents_df.show(5)

+--------------------+--------------------+--------------------+-------------+-------------+----------+-------+--------------------+---------+--------------------+--------------------+--------------------+------+
|            casebody|           citations|               court|decision_date|docket_number|first_page|     id|        jurisdiction|last_page|                name|   name_abbreviation|            reporter|volume|
+--------------------+--------------------+--------------------+-------------+-------------+----------+-------+--------------------+---------+--------------------+--------------------+--------------------+------+
|{{[Charles J. Gra...|[{185 Ill. App. 3...|{8837, NULL, Illi...|   1989-07-13|No. 4-88-0757|       566|2646945|{29, Ill., Illino...|      570|In re MARRIAGE OF...|In re Marriage of...|{Illinois Appella...| {185}|
|{{[Fellheimer, Tr...|[{185 Ill. App. 3...|{8837, NULL, Illi...|   1989-07-13|No. 4-88-0897|       552|2647072|{29, Ill., Illino...|      560|THE PE

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, lower, regexp_replace, col
from pyspark.sql.types import ArrayType, StringType
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Make sure NLTK datasets are downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abhishekshah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishekshah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

24/04/08 17:32:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
# Define stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Adjusted preprocess_text function for Spark UDF compatibility
def preprocess_text(text):
    # Tokenization
    words = word_tokenize(text.lower())
    # Removing stopwords and stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words and word.isalpha()]
    return words

# Register the UDF
preprocess_text_udf = udf(preprocess_text, ArrayType(StringType()))


In [8]:
from pyspark.sql.functions import explode

# Explode the opinions array to work with individual opinions
exploded_docs_df = documents_df.withColumn("opinion", explode("casebody.data.opinions"))

# Apply preprocessing to the 'text' field of each opinion
preprocessed_docs_df = exploded_docs_df.withColumn("words", preprocess_text_udf(col("opinion.text")))

# Show the structure and some example processed data
preprocessed_docs_df.printSchema()
preprocessed_docs_df.select("words").show(truncate=False)

root
 |-- casebody: struct (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- attorneys: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- head_matter: string (nullable = true)
 |    |    |-- judges: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- opinions: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- author: string (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- parties: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- status: string (nullable = true)
 |-- citations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cite: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- court: struct (nullable = true)
 |   

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
from pyspark.sql.functions import explode, col

# Using the column name 'id' as the document identifier
try:
    # Explode the 'words' column into separate rows and select the 'id' column as the document identifier
    flat_words_df = preprocessed_docs_df.withColumn("word", explode(col("words"))).select(col("id").alias("doc_id"), "word")

    flat_words_df.show(truncate=False)  # For debugging, to see if it works as expected
except Exception as e:
    print(f"Encountered an error: {e}")


+-------+--------+
|doc_id |word    |
+-------+--------+
|2646945|justic  |
|2646945|lund    |
|2646945|deliv   |
|2646945|opinion |
|2646945|court   |
|2646945|respond |
|2646945|carol   |
|2646945|neill   |
|2646945|appeal  |
|2646945|trial   |
|2646945|court   |
|2646945|order   |
|2646945|dispos  |
|2646945|marit   |
|2646945|properti|
|2646945|determin|
|2646945|mainten |
|2646945|child   |
|2646945|support |
|2646945|payment |
+-------+--------+
only showing top 20 rows



In [10]:

from pyspark.sql.functions import collect_list

# Group by 'word' and aggregate the document IDs into a list
inverted_index_df = flat_words_df.groupBy("word").agg(collect_list("doc_id").alias("doc_ids"))

inverted_index_df.show(truncate=False)


+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:

from pyspark.sql.functions import count

# Assuming 'flat_words_df' contains columns 'doc_id' and 'word'
doc_lengths_df = flat_words_df.groupBy("doc_id").agg(count("word").alias("doc_length"))

doc_lengths_df.show(truncate=False)



+-------+----------+
|doc_id |doc_length|
+-------+----------+
|2657478|1515      |
|2667976|1629      |
|2658397|721       |
|2654304|1249      |
|2665806|1290      |
|2655741|972       |
|2659369|2752      |
|2656116|1065      |
|2680546|993       |
|2655906|3878      |
|2664578|1563      |
|2668171|1040      |
|2657393|1696      |
|2669314|1709      |
|2663704|1244      |
|2658352|2564      |
|2657433|1032      |
|2666367|832       |
|2655870|1688      |
|2666410|1090      |
+-------+----------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import count

# Adjusting the number of partitions to better match the MacBook's capabilities
# The M1 chip has 8 cores, but considering it's a single machine, 
# a lower number might prevent excessive context switching and memory overhead.
# Start with a number like 8 and adjust based on performance and resource utilization.
num_partitions = 8

# Repartition the DataFrame before performing the groupBy operation.
# This can help optimize the parallelism and memory usage.
repartitioned_flat_words_df = flat_words_df.repartition(num_partitions, "doc_id")

# Calculate term frequency (TF) for each word in each document
term_frequencies_df = repartitioned_flat_words_df.groupBy("doc_id", "word").agg(count("word").alias("term_freq"))

# Display the calculated term frequencies. Adjust the number of rows shown as needed.
term_frequencies_df.show(truncate=False)


+-------+--------+---------+
|doc_id |word    |term_freq|
+-------+--------+---------+
|2646945|justic  |1        |
|2646945|lund    |1        |
|2646945|deliv   |1        |
|2646945|opinion |5        |
|2646945|court   |17       |
|2646945|respond |11       |
|2646945|carol   |1        |
|2646945|neill   |2        |
|2646945|appeal  |2        |
|2646945|trial   |10       |
|2646945|order   |2        |
|2646945|dispos  |1        |
|2646945|marit   |15       |
|2646945|properti|10       |
|2646945|determin|4        |
|2646945|mainten |1        |
|2646945|child   |3        |
|2646945|support |4        |
|2646945|payment |6        |
|2646945|attorney|4        |
+-------+--------+---------+
only showing top 20 rows



In [13]:

from pyspark.sql.functions import log10, countDistinct, col

# Calculate the total number of documents
total_docs = doc_lengths_df.count()

# Since repartition can cause extensive shuffling, especially before a groupBy operation,
# ensure that it's necessary based on your dataset's characteristics and size.
# Adjust the number of partitions if you decide to repartition.
repartitioned_flat_words_df = flat_words_df.repartition(8, "word")

# Calculate the document frequency (DF) for each term: the number of documents containing the term
doc_frequency_df = repartitioned_flat_words_df.groupBy("word").agg(countDistinct("doc_id").alias("doc_freq"))

# Calculate the IDF for each term
idf_df = doc_frequency_df.withColumn("idf", log10(total_docs / col("doc_freq")))

idf_df.show(truncate=False)


+---------+--------+--------------------+
|word     |doc_freq|idf                 |
+---------+--------+--------------------+
|respond  |438     |0.3585258894959005  |
|determin |819     |0.08671609823958157 |
|section  |677     |0.1694113313148557  |
|rev      |837     |0.07727454200674004 |
|charg    |489     |0.3106911408763798  |
|attempt  |484     |0.3151546383555875  |
|two      |762     |0.1180450286603995  |
|parti    |682     |0.16621562534352108 |
|limit    |515     |0.28819277095880896 |
|period   |394     |0.4045037781744259  |
|separ    |354     |0.4509967379742122  |
|follow   |814     |0.08937559511079875 |
|action   |644     |0.1911141326401879  |
|ad       |356     |0.44855000202712486 |
|consist  |378     |0.4225082001627747  |
|date     |460     |0.3372421683184259  |
|also     |896     |0.047691990337874815|
|valu     |223     |0.6516951369518393  |
|similarli|223     |0.6516951369518393  |
|made     |749     |0.12551818230053352 |
+---------+--------+--------------

In [14]:

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def preprocess_query(query):
    # Convert to lowercase
    query = query.lower()
    
    # Tokenize the query
    tokens = word_tokenize(query)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Stem the tokens
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    return stemmed_tokens



In [15]:
from pyspark.sql import functions as F

# Pre-calculate avgdl
total_doc_length = doc_lengths_df.agg(F.sum("doc_length").alias("total")).collect()[0]["total"]
avgdl = total_doc_length / doc_lengths_df.count()


In [16]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType

def calculate_bm25(term_freq, doc_length, avgdl, idf, k1=1.2, b=0.75):
    return idf * (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * (doc_length / avgdl)))

calculate_bm25_udf = udf(calculate_bm25, DoubleType())


In [17]:
# Example query
query = "Murder"
query_terms = preprocess_query(query)
print(query_terms)

['murder']


In [18]:
print(f"Average document length (avgdl): {avgdl}")

Average document length (avgdl): 1530.152


In [19]:

# Filter term_frequencies_df for the query terms
filtered_term_freqs_df = term_frequencies_df.filter(term_frequencies_df.word.isin(query_terms))

# Join with idf_df to get IDF values
term_freqs_idf_df = filtered_term_freqs_df.join(idf_df, on="word", how="left")

# Join with doc_lengths_df to get document lengths
term_freqs_idf_lengths_df = term_freqs_idf_df.join(doc_lengths_df, on="doc_id", how="left")

# Calculate BM25 score for each term
scored_df = term_freqs_idf_lengths_df.withColumn("bm25_score", calculate_bm25_udf(col("term_freq"), col("doc_length"), F.lit(avgdl), col("idf")))

# Sum BM25 scores for each document
result_df = scored_df.groupBy("doc_id").agg(F.sum("bm25_score").alias("total_score"))

# Show top N documents
result_df.orderBy(col("total_score").desc()).show()

# Extract text from the "opinion" column
opinion_text_df = exploded_docs_df.selectExpr("id as doc_id", "opinion.text as opinion_text")

# Show the resulting DataFrame
opinion_text_df.show(truncate=False)

+-------+------------------+
| doc_id|       total_score|
+-------+------------------+
|2678660|1.8393128455016596|
|2507291|1.8302167417701507|
|2518462|1.8232122630571204|
|5589302| 1.821690463456325|
|2510565|1.8169394377485508|
|2513497| 1.815765765038252|
|2513450|1.8150281879198804|
|2688941| 1.813490952963405|
|2679363|1.8074717225104366|
|2507775|1.8026210070540254|
|2511324|1.8018403727141574|
|2510630|1.7997040741680805|
|2690530|1.7912841445000593|
|2519449|1.7911541723751725|
|2678599|1.7813354033406428|
|5569514|1.7779291760531442|
|3260348|1.7706310973949775|
|2509831|1.7683515070603162|
|5567120| 1.764056418896808|
|2680602|1.7609735028218263|
+-------+------------------+
only showing top 20 rows

+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
# Save all dataframes to individual parquet files and all values to a single csv file
# # Save the preprocessed documents DataFrame
preprocessed_docs_df.write.mode("overwrite").parquet("preprocessed_docs.parquet")

# # Save the flat words DataFrame
flat_words_df.write.mode("overwrite").parquet("flat_words.parquet")

# # Save the inverted index DataFrame
inverted_index_df.write.mode("overwrite").parquet("inverted_index.parquet")

# # Save the document lengths DataFrame
doc_lengths_df.write.mode("overwrite").parquet("doc_lengths.parquet")

# # Save the term frequencies DataFrame
term_frequencies_df.write.mode("overwrite").parquet("term_frequencies.parquet")

# # Save the IDF values DataFrame
idf_df.write.mode("overwrite").parquet("idf_values.parquet")

# # Save the BM25 parameters needed for scoring
scoring_params_df = spark.createDataFrame([(avgdl,)], ["avgdl"])
scoring_params_df.write.mode("overwrite").parquet("scoring_params.parquet")

#save the opinion text DataFrame
opinion_text_df.write.mode("overwrite").parquet("opinion_text.parquet")

24/04/08 18:01:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
